Here are summarization notes I completed from the book "Speech and Language Processing" by Jurafsky and Martin

Specifically Chapters 2-11 encapsuling Part I of the textbook which covers Fundamental Algorithms for NLP.

Notes: 
    
    1. Some of the lessons that I read through I did not include in these notes, these were on topics that I am already very familiar with and didnt find it worth my time creating notes for.
    2. Bodies of text surrounded by quotations ("") are direct quotes from the textbook that I copied over as I really liked the wording of the explanation.  

# Chapter 2

"I do uh main- mainly business data processing" main- is fragment, uh is a filler

Can filter out these disfluencies for speech transcriiption system but these can be left in if doing next word prediction as usually means person is starting a new idea in sentence.

Herdan’s Law (Herdan, 1960) or Heaps’ Law (Heaps, 1978): |V| = kN^β (the amount of unique words in a corpus based on the amount of words (N) (rest are constants)

**lemma** is a set of lexical forms having the same stem, the same
wordform major part-of-speech, and the same word sense. 

**Wordform** is the full inflected
or derived form of the word. The two wordforms cat and cats thus have the same
lemma, which we can represent as cat

**Clitic** is a part of a word that can’t stand on its own, and can only
occur when it is attached to another word. Some contractions occur in other alphabetic languages, including articles and pronouns in French (j’ai, l’homme). another ex: They're (re cant hold seperatly)

often want
to keep the punctuation that occurs in words internally, in examples like m.p.h., Ph.D.,
AT&T, and cap’n. Special characters and numbers will need to be kept in prices
($45.55) and dates (01/02/06); we don’t want to segment that price into separate
tokens of “45” and “55”. And there are URLs (https://www.stanford.edu),
Twitter hashtags (#nlproc), or email addresses (someone@cs.colorado.edu)

Penn Treebank tokenization standard: used for the parsed corpora (treebanks) 

separates out clitics (doesn’t becomes "does" + "n’t")

In [ ]:
text = "That U.S.A. poster-print costs $12.40..."
pattern = r"""(?x) # set flag to allow verbose regexps
#(?:[A-Z]\.)+ # abbreviations, e.g. U.S.A.
| \w+(?:-\w+)* # words with optional internal hyphens
| \$?\d+(?:\.\d+)?%? # currency, percentages, e.g. $12.40, 82%
| \.\.\. # ellipsis
 | [][.,;"’?():_‘-] # these are separate tokens; includes ], [
 """
nltk.regexp_tokenize(text, pattern)
#[’That’, ’U.S.A.’, ’poster-print’, ’costs’, ’$12.40’, ’...’]


To deal with this unknown word problem, modern tokenizers automatically insubwords duce sets of tokens that include tokens smaller than words, called subwords. Subwords can be arbitrary substrings, or they can be meaning-bearing units like the
morphemes -est or -er. (A morpheme is the smallest meaning-bearing unit of a language; for example the word unlikeliest has the morphemes un-, likely, and -est.)
In modern tokenization schemes, most tokens are words, but some tokens are frequently occurring morphemes or other subwords like -er. Every unseen word like
lower can thus be represented by some sequence of known subword units, such as
low and er, or even as a sequence of individual letters if necessary.

Tokenization techniques: 

- token learner: turns word seperated training corpus into vocab set and token set

- token segmenter: The token segmenter takes a raw test sentence and segments it into the
tokens in the vocabulary


: byte-pair encoding
(Sennrich et al., 2016), and unigram language modeling (Kudo, 2018), There is
also a SentencePiece library that includes implementations of both of these (Kudo
and Richardson, 2018a), and people often use the name SentencePiece to simply
mean unigram language modeling tokenization.

***Byte-pair encoding***: takes training corpus and splits into indiv chars. Then takes 2 most frequent adjacent chars and groups them together everywhere in the corpus. Ex: finds ('a', 'b') and merges into 'ab'. 

"It continues to count and merge, creating
new longer and longer character strings, until k merges have been done creating
k novel tokens; k is thus a parameter of the algorithm. The resulting vocabulary
consists of the original set of characters plus k new symbols"

useful vocab that it finds is the sequence "er " which it can figure out the meaning of when applied to new words. 
Learns "er " from "newer" and learns "low" now can figure out what "lower" means. 
Note: This alg is run many thousands of times and will fully tokenize most words, will only split a word that is very rare (it hasnt seen before in the training corpus)

***Lemmatization***: finds the root of several words. (Ex: am, are, and is have the shared lemma be; the words dinner and dinners
both have the lemma dinner)

Morphology: study of how words are built up from smaller meanings subwords called morphemes

***stems***: the central moraffix pheme of the word, supplying the main meaning

***affixes***: adding “additional” meanings of various kinds. 

Ex: the word fox consists of one morpheme
(the morpheme fox)

cats consists of two: the morpheme cat and the
morpheme -s

***Stemming***: Naive form of Lemmatization. Has set of rules for simplifying endings of words (ex: ational -> ate "relational -> "relate", ing -> none)

***Minimum edit distance***: as the minimum number of editing operations (operations like insertion, deletion, substitution) needed to transform one string into another.
The gap between intention and execution, for example, is 5 (delete an i, substitute e for n, substitute x for t, insert c, substitute u for n). Diff weighting values can be assigned for diff operations (Levenshtein distance just uses 1 for each type)
Can represent this as a trie problem that uses dynamic programming to find shortest amount of operations to a word (dynamic programming to reduce repeated children)

# Chapter 3: N-Grams

P(the|its water is so transparent that) = C(its water is so transparent that the) / C(its water is so transparent that)
C() = count the amount of occurnaces in corpus, P() = probability

P(xn | x1, x2, ..., xn-1) = P(x1)*P(x2|x1) * P(x3|x1, x2) * P(x4 | x1, x2, x3), etc..

* Isnt that good as kind of creative and new sentence. Could do P(the | marry popps, the water is so transperant that ) and would find zero instances

***Bi-gram model***: instead of joint prob of entire sequence of words only use the last two words so for "the water is so transparent that) return P(the | that) , p(some_other_word | that), etc... This is known as a way of approximating the full sequence prob. For bi-gram is aka Markov Model

Also exists ***Tri-gram model*** and so on (can use N var to explain as ***N-gram model***


Thus for entire sequence (or sentence) we can approx the probability of it occuring P(w_1:w_N) ~= P(w2|w1)*P(w3|w2)*P(w4|w3)*...*P(wN|wN-1)

To get P(w2|w1) use ***MLE (maximum likelihood estimation)*** get count of occurances of two words that contain "wn-1 wn" and divide them by the amount of all two word combos that contain "wn-1 x" (x is any word). So P(w2|w1) = C("wn-1 wn") / C("wn-1 x") (note this can be simplified to just put C("wn-1") in the bottom as we know that each word occurance of wn-1 will have a word that follows it. thus the same count as its 1 occurance)

Ex:

t: I am Sam :t

t: Sam I am :t

t: I do not like green eggs and ham :t

P(I|t:) = 2/3 = .67 

P(Sam|t:) = 1/3 = .33


***Extrinsic evaluation***: best method for eval LM - add langiage model inside your app and see if it improves your product/results

***Intrinsic evaluation***: quick and cheap eval of LM - use some metric to eval performance and compare models (ex: Perplexity)

***Perplexity***: the inverse probability of the test set (one over the probability of the test
set), normalized by the number of words. . For a test set W = w1w2 ...wN: P(W) = P(w1,w2, ..., wN)^(1/N) = P(w1, w2, ..., wN) = P(w1|sentence_pad) * P(w2 | w1, sentence_pad) * P(w3 | w2, w1) etc.. (then take inverse of this) Meaning we want higher probs of it predicting w3 if given w2 and w1 in the test set. Take the inverse of this and we want perplexity (this inverse) to be low. Meaning that our model is able to have good probability when predicting the next words given some previous words in the test set. \

"the higher the probability of the word sequence, the lower the perplexity. 
Thus the lower the perplexity of a model on the data, the better the model, and minimizing perplexity is equivalent to maximizing the test set probability according to the language model."

*Note: to have an accurate perplexity it should eval on this test set not knowing the vocabulary in it beforehand (essentially brand new data). This way we can see how generally smart the model is.

"An (intrinsic) improvement in perplexity does not guarantee an (extrinsic) improvement in the performance of a language processing task like speech recognition or machine translation."

Sampling sentences from a language model (by creating distribution of each word and a range representing its frequency. Then selecting random likelihoods and selecting the word that has that liklihood in its range) will create a random sentence that represents the general knowledge or the words it knows (will most likely see the more common words but have chance to see more rare words thus representing your corpus distribution)

Using larger N-gram models will create better samples as sampling unigram model will sample one random word each time, but sampling from a 4-gram model will sample 4 words each time which will be actual 4 word sequences and make sense.

Ex: 

unigram: "he going when lunch the to"

4-gram: "I am going to when" ("I am going to" is the 4 gram sampled which is a perfect 4 word structure)

***Unseen sequences in test set***: can use smoothing or discounting

***Unseen words in test set***: can use closed vocabulary set (making sure test set doesnt have any words that arent in training set) or if using tokens instead of words can use BPE algorithm and split new unseen word into subword tokens or character tokens (kind of like how we sound out words by letters when we havent seen it before when reading)

"One way to create an open vocabulary system is to model potential open
vocabulary
unknown words in the test set by adding a pseudo-word called </UNK/>"

***Laplace smoothing***: add 1 to all the ngram counts so each word has a count of at least 1 (does not perform well enough to be used in modern n-grams. Since in whatever n-gram word combos there are a lot of combos that dont exist thus need to take alot of counts away from combos that do exist, thus significantly lowering their probabilities)

***Discounting***: removing some counts from words and adding them to the zero counts. 

***Add-k smoothing***: instead of adding 1 adds some fraction to each count (like 0.5 for example). Although this still doesnt work well in practice (can be useful for some like text-classification).

***Backoff (aka stupid backoff)***: If trying to get prob for tri-gram p(w3|w2, w1) but "w1 w2 w3" doesnt exist in training corpus we can 'back off' and try bi-gram: p(w3|w2) "w3 w2", and if that doesnt work we can 'back off' again and try unigram: p(w3) "w3".

***Interpolation***: Like backoff but instead we use combination of all sub-gram probs. For ex in ***linear interpolation*** we do x3*P(w3|w2 w1) + x2*P(w3|w2) + x1*P(w3). x1 values can be preset constant values or create them based on the counts of the subgram count. if tri-gram doesnt exist can create x3 based on the count of C(w3, w2) in a validation set using an algorithm like the ***EM Algorithm***: (iterativly converges on locally optimal xi values)."

***Katz backoff***: when backoff to a sub-gram we discount the probability we give to the zero higher-gram parts to create a more even n-gram and sub-n-gram distribution. "Katz backoff is often combined with a smoothing method called Good-Turing."


# Chapter 4 - Naive Bayes


Skipping over the Naive bayes notes as very familiar with standard ML algorithms already

Statistic Significance testing: If certain metric of algorithm A is better than algorithm B perform see if it was more likely that it was only due to random chance (the difference isnt large enough to conclude that it will be better in every scenerio). For NLP dont normally use basic tests like t-test or ANOVA. As these tests make basic assumptions about our data (like normally distributed for example) which isnt often the case for NLP. Instead create thousands of samples and combine the results to check. Common methods include approximate randomization and the bootstrap test.

"***Paired tests*** are those in which we compare two sets of observations that are aligned"

***Paired Bootstrapping***: "bootstrapping refers to repeatedly drawing large numbers of samples with replacement (called bootstrap samples) from an original set". Randomly sample from dataset with the chance of sampling an item more than once. In paired we test model A and model B on classifying the same boostrapped sampled data and compare the results between the two models. We then create a norm value between 0 and 1 of the amount of times the metric of method A surpased method B by a certain margin divided by the amount of trials.





# Chapter 5 - Logistic Regression

Skipping over as Logisitic regression and gradient descent review

# Chapter 6 - Vector Semantics and Embeddings

***Word Sense***: One word can have multiple meanings (ex: mouse can mean the rodent or the cursor control device for the computer)

***wordforms***: The extended form of a lemma word. (ex: lemma of sing has wordform sang (past tense of sing)

***Word Relatedness***: Words can be related without having similar meanings. For ex: Coffee and Cup.

***Semantic Field***: A set of words which spread across a certain ("semantic domain") or topic. For example: the semantic field of hospitals (surgeon, scalpel, nurse, anesthetic, hospital). "Semantic fields are also related to topic models, like Latent Dirichlet Allocation, LDA, which apply unsupervised learning on large sets of texts to induce sets of associated words from text."

***Semantic Frame***: Similar to a semantic field, a semantic field represents an event or scenerio. "set of words that denote perspectives or participants in a particular type of event". For example: A scenerio of two people buying and selling one thing. You cant represent it using just the words "buy" and "sell", each word represents a different perspective and you have to group them to describe one scenerio happening involving different people. 

"Frames have semantic roles (like buyer, seller, goods, money), and words in a sentence can take on these roles. Knowing that buy and sell have this relation makes it possible for a system to know that a sentence like "Sam bought the book from Ling" could be paraphrased as "Ling sold the book to Sam", and that Sam has the role of the buyer in the frame and Ling the seller."

***Conotation***: How a word represents the mood of the writer of the word. For example: wonderful vs dreary. "consider the difference in connotations between fake, knockoff, forgery, on the one hand, and copy, replica, reproduction on the other, or innocent (positive connotation) and naive (negative
connotation)."

***Sentiment***: "Positive or negative evaluation language" 

"
***valence***: the pleasantness of the stimulus
***arousal***: the intensity of emotion provoked by the stimulus
***dominance***: the degree of control exerted by the stimulus
Thus words like happy or satisfied are high on valence, while unhappy or annoyed are low on valence. Excited is high on arousal, while calm is low on arousal.
Controlling is high on dominance, while awed or influenced are low on dominance.
"

Assigning a numeric value for those 3 terms for each word gives you a vector that you can use to represent words in 3d space.

***Vector Semantics***: Standard way to represent word meaning in NLP.

***Embeddings***: Vectors for representing words.

***Term-document matrix***: Each row represents a word, each column represents a document from some collection of documents.

***Information retrieval***: using a query vector q to find the best matching document d from a collection of documents D.

***Term-term matrix (aka: word-word matrix, term-context matrix)***: Each row is a target word, and each column is a (context) word. Thus is a matrix that counts the amount of times two words occur in a document. It can count overall or the intersection of two words in a NxN window (-n words left, n words right, -n words above, and n words below). Ex: apple x pie, sugar x cherrie each have higher counts as when see apple in a sentence have high chance to see in either previous sentence or next sentence.

***Dot product great measure for similarity*** as results in high value when vectors have high values in the same dimension. But if have values in different dimensions this will be orthogonal and have a dot product of zero creating a zero similarity value. 

Issue with dot product is that it favours longer vectors thus more frequent words. For similarity we want to only check similarity regardles of words frequency. To fix this we normalize the vector length by dividing the dot product by the lengths of each of the two vectors. This is the same value as the cosign angle between two vectors, thus creating ***cosign similarity***. Can calculate the similarity using the word counts in a document (the similarity of two words from a ***term-term matrix***). This is known as using co-occurance matricies using word frequency to represent word similarity. aka: Raw frequency.

Issue with raw-frequency is that it has skewed high similarity values for all words with stop words like "the" and "it" for example. Words that co-occur once or twice are not important/significant but words that co-occur too many times are also not important like "the" and "good". So need to find balance between. Ex: "pie" and "cherry" more important similarity than "the" and "pie" even though "the" and "pie" occur more frequently together than "pie" and "cherry".

***TF-IDF (Term Frequency - Inverse Document Frequency)***:
    - ***Term Frequency***: The frequency of the word t in the document d. tf_t,d = count(t, d) (can also use log_10_(count() instead to squash frequency a bit)
    - ***Document Frequency***: df_t of a term (word) t is the number of documents a word occurs in. This helps us seperate between the terms/words that appear in only a few documents (like "pie") versus words that appear in every document (like "the").
    - ***Inverse Document Frequency***: N/df - Total number of documents in the collection (N) / document frequency of term (df_t). This emphasises the importance of words that only appear in a few documents as the fewer documents they appear in the smaller N becomes thus our IDF has a larger value meaning the TF-IDF word importance will be higher.

TF-IDF is often used for document functions like comparing two documents to see if they are similar.

***Collection frequency***: the total amount of times a term/word appears in a whole collection of any document.

"Consider in the collection of Shakespeare’s 37 plays the two words Romeo and action. The words have identical ***collection frequencies*** (they both occur 113 times in all the plays) but very different ***document frequencies***, since Romeo only occurs in a single play. If our goal is to find documents about the romantic tribul/ations of Romeo, the word Romeo should be highly weighted, but not action."

***PPMI (Positive Pointwise Mutual Information)***: Used for ***term-term matricies** - "PPMI draws on the intuition that the best way to weigh the association between two words is to ask how much more the two words co-occur in our corpus than we would have a priori expected them to appear by chance." PMI(word, context(other word)) = log_2( P(w, c) / (P(w)*P(c)) ). Numerator (P(w, c)): How often we observed the two words together. Denominator (P(w)*P(c)): How often we expected to observe the words together assuming they occur independently. Values can range from negative to positive thus we can have a negative value for words that occur together less often than we expect them to. However, negative correlation is an unreliable statistic for datasets that arent enormous and even then we need human interpretation for each sample to evaluate if the info is useful. Thus we convert the negative values to zero for PPMI = max(PMI, 0).

\*Note: PMI is biased towards events that dont occur often. Ex: rare words (based on corpus like a computer based textbook) like "cherry" and "pie" will haved a higher PMI value than "data" and "computer". To combat this we modify the PMI formula to use Probability functions raised to the power of a const value alpha: P_alpha(c) = count(c)^alpha / sum_c count(c)^alpha. Setting an alpha value to something like alpha=0.75 works as it will reduce the higher PMI values than it will reduce the lower PMI values. (Can also use laplace smoothing).\*

***Centroid***: A multidimensional version of a mean. Used to represent the mean value of a document using a ***centroid document vector***: Given k word vectors w1,w2,...,wk;  d = w1,w2,...,wk / k. Then with two centroid document vectors d1 and d2 you can compare their similarity using cosign similarity: cos(d1, d2). Useful for things like: information retrieval, plagiarism detection, and news recommender systems.

"Either the PPMI model or the tf-idf model can be used to compute word similarity, for tasks like finding word paraphrases, tracking changes in word meaning, or automatically discovering meanings of words in different corpora. For example, we can find the 10 most similar words to any target word w by computing the cosines between w and each of the V −1 other words, sorting, and looking at the top 10"


***Word2Vec***: embeddings words as short dense vectors instead of large sparse vectors like is done in term-document matrix. These embeddings are much smaller than sparse vectors and result in better performance in every NLP task. (One reason for this is becuase it requires a model to learn much fewer weights to use the word vectors. The smaller size also helps with generalization and avoid overfitting

"The intuition of word2vec is that instead of counting how often each word w occurs near, say, apricot, we’ll instead train a classifier on a binary prediction task: “Is word w likely to show up near apricot?” We don’t actually care about this prediction task; instead we’ll take the learned classifier weights as the word embeddings."

***static embeddings***: model learns a fixed embedding for each word.

***self-supervision***: Avoids the need for a seperate training set for the binary classifier for word2vec. Instead, we update and train the model as it runs. Each next word is the target word value that we need to predict (for binary classifier in word2vec: "is this word x likely to appear next to apricot?" or for Neural net: predict target word x) and we tweak the model weights to adjust its prediction for that next word and continue this process for the entire future stream of words. We can then take the weights created as the embedding vector for each word as it runs.

***Skip-gram***: Intuition:

    1. Treat the target word and a neighboring context word as positive examples.
    2. Randomly sample other words in the lexicon to get negative samples.
    3. Use logistic regression to train a classifier to distinguish those two cases.
    4. Use the learned weights as the embeddings

***Inner workings of the classifier***: 

    1. Goal is to create a classifier that given two words ((w, c) -> (target_word, context_word)) will return the probability that w and c actually belong together. Ex: P(+| w, c): P(+| apricot, jam) = 0.75 vs P(+|apricot, computer) = 0.11
    2. The similarity probability function uses the dot product on the dense word embeddings to check for similairty (since two vectors are similar if they result in high dot product value (since they have high values in the same dimensions multiplied by each other) similarity(w, c) ~= w*c (This is just cosign similiarty without the normalization)).
    3. Now we have a similarity value ranging from (-inf, inf) and we turn it into a probability by using a sigmoid function: sigmoid(x) = 1/(1+exp(-x))
    4. Thus the probability that two words are together is P(+| w, c) = 1/(1+exp(-w*c))
    5. To get the probability that the two words dont belong together we just subtract 1: P(-| w, c) = 1 - P(+| w, c) = 1 - sigmoid(w*c) = sigmoid(-w*c) = 1/(1+exp(w*c)). Therefore  P(-| w,  c) = 1/(1+exp(w*c)) (All we need to do is remove the negative sign for the embeddings dot product)
    6. Now we apply this classifier to a set window of words wrapping a target word. Ex: "...lemon, a [tablespoon of apricot jam, a] pinch..." -> [c1, c2, w, c3, c4]. We get the probability for this window being real by doing P(+| w, c_1:L) = sig(w, c1) * sig(w, c2) * ... * sig(w, cL) (Note by doing this we are making the assumption that each context word is independent of the other context words and checking if it belongs beside w). We further simplify this by taking the log of this prob window to get: log(P(+| w, c_1:L)) = L_sum_i=1 sig(w, ci). This returns a probability of how similar the context window (c1:L) is to a given target word (w).

***Skip-gram embedding weights***: For skip-gram embedding we learn a matrix of size 2|V| x d (with V as our vocab count) and d as the vector size of each words dense embedding. So we have 2 |V| x d matricies contatinated. This is becuase we have 2 seperate embeddings for each word: one for the word as a context word and one for the word as a target word.

***Classifier Training process***:

    1. Start by initializing 2 |V| x d matricies. One called matrix W (for the weights of each word as a target) and one called matrix C (for the weights of each word as a context word).
    2. Then start with a window to sample context and a target word. For the current window you generate one target word (w) and L context words (c1:L) thus the following positive training samples: (w, c1), (w, c2), ..., (w, cL). At the same time we generate negative samples where we sample a word randomly from a lexicon that doesnt include our target word. We sample much more negative samples than positive samples at a rate of (a preset) 1:k ratio. Ex: positive: (apricot, jam), (apricot, tablespoon); negative: (apricot, computer), (apricot, aardvark), (apricot, mushroom), (apricot, poland) (here k = 2 at a 2:4 ratio of positive to negative samples)
    3. Minimize a loss function like so: L_CE = -log[ P(+| w, c_pos) k_prod_i=1 P(-| w, c_neg_i) ] (leaving out simplified formula). To minimize this loss function we have to maximize the similarity of the target word and a positive context word and minimize the similarity of the target word and the sampled negative words.
    4. Take the derivative of this loss function then perform stochastic gradient descent that will update word embeddings for positive and target words to be more similar and negative and target words to be further apart.
    5. Note: We learn two seperate embeddings for each word (wi and ci; target embedding and context embedding). Here we can just add them together wi+ci or even just throw away the C (context matrix) altogether.


***fasttext***: Solves problem that word2vec faces where model doesnt receivce enough training data on rare words. Combats this by splitting words into n-grams. Ex for n=3 "where" gets represented as: \<where>\<and>\<wh, whe, her, ere, re>. Now unknown words can be represented by summed n-grams.


When we train a model the size of our context window determines our results. 

A short window, something like +-2 words (on each side) will have the model return semantically similar words with the same parts of speech. Words from this type of result are typically known as ***first-order co-occurrence***. 

A long window, will return words that are topically related but not similar. This will result in words that have similar neighbors but are not usually similar, known as ***second-order co-occurrence***

# Chapter 7 - Neural Networks and Neural Language Models

\*Skipping most of notes here as just explaining the inner workings of a Neural Network*

When inserting multiple words into a neural network where each word is represented as a dense embedding there are several sstrategies (known as ***embedding pooling***:

    - Sum all the word embeddings and divide by the amount of words (word embedding mean value)
    - For each dimension of the word embedding take the max value abailable (ex: [(0, 1, 2, 0), (2, 0, 3, 0)] -> (2, 1, 3, 0))

For smaller NLP tasks n-grams are prefered due to their simplicity, fast speed, and interprability.

For all tasks though Neural Networks outperform n-gram models but they are more complex, large, and black-box models. So they should only be used for NLP tasks with large datasets.

# Chapter 8 - Sequence Labeling for Parts of Speech and Named Entities

***Named Entity***: Anything that we can refer to with a name like: a person, a location, an organization.

***POS (parts of speech)*** and named entities are useful for predicting sentence structure and meaning. An example of one knowable pattern is: (nouns in English are preceded by determiners and adjectives, verbs by nouns).

POS Types:
    ***Closed class***: classes of words that have a "fixed membership" meaning their arent new words being added. These are classes like prepositions or function words. Ex: There is a fixed set of function words ("of", "it", "the") (new ones arent being created).
    ***Open class***: classes of words that are constantly being created or borrowed. For example, nouns like "Iphone" and "to fax". Other open classes include: verbs, adjectives, adverbs, and interjections.

***Proper nouns***: Names of specific persons or entities. Ex: Regina, Colorado, and IBM.

***Adjective***: Describes a property or quality of a noun like colour (white), age (old/young), and value (good/bad).

***Adverb***: Describes or modifies a verb. 
 
    "He sings loudly". 
***Directional/locative adverb***: words that specify the location or direction of a verb. Ex:

    "I ran *home extremely quickly yesterday*" (words between ** are all adverbs. home is a directional/locative adverb)
    
***Degree adverb***: Specifies the degree or intensity of a verb. For example "extremely" in the example above.

***Manner adverb***: Describes how a verb happend (or the manner of some action). For example, "quickly" in the above example.

***Temporal adverb***: Describes the time a verb happened. For example, "yesterday" in the above example.

***Interjections***: Small almost filler words. Ex: "oh, hey, alas, uh, um, hello, goodbye"

***Preposition***: Usually describe some spacial or temperal relation to a word. The word means they have to sit before the noun they are placed to pre-position. Ex: (above the tabel, below the table, on the table). "shows direction, time, place, location, spatial relationships, or to introduce an object. Some examples of prepositions are words like 'in,' 'at,' 'on,' 'of,' and 'to.', (on it, before then, by the house), (on time, with gusto, beside herself)"

***Particle***: Words that dont fit into any other class of speech (the left over classification of a word). For example "Jack cut off the limb". Off is a particle as its not part of the verb cut and its not an adverb as it doesnt really change the intensity of cutting. Another example is the infinitive 'to' in 'to fly', although it can also act as a preposition, e.g. 'I'm going to Spain next week' it means something different in 'to fly' than it does in the other statement.

***Non-compositional***: means a words gramatical class cant be predicted without context. The individual word cant be classified.

***Determiner***: Used to modify, introduce, or describe nouns or noun-like words. They also mark the start of an english noun phrase. Words like this and that. "Determiners help clarify what a noun is referring to and are typically placed before descriptive adjectives. For example, in the sentence Would you like to buy this new book?" 

***Article***: Subcategory of a Determiner that refers to a specific instance of an entity. Ex: the, a, an

    Article Example: "I recommend you watch *a* video first", "I recommend you watch *the* video first"
    Determiner example: "I recommend you watch *that* video first", "I recommend you watch *this* video first"

***Conjunction***: Word that joins two phrases like and, or, and but. Ex: "conjunction \*that* in “I thought that you might like some milk” links the main clause I thought with the subordinate clause you might like some milk."

***Pronouns***: Shorthand variable word that refers to an entity or event. Examples like: you, she I, it, me, they, them.

***Possessive pronouns***: Indicates a possesion or a relation between a person and an object. Ex: my, your, his, her, its, one’s, our, their.

***Wh-pronouns***: pronouns that are used in question forms. Ex: what, who, whom, whoever, "Frida, who married Diego..."

 ***Named Entity Recognition (NER)***: "assigning words or phrases tags like PERSON, LOCATION, or ORGANIZATION". Useful as a first step in a lot of NLP tasks. For example, performing sentiment analysis of a customer review of a particular entity (product).

     Ex: "Citing high fuel prices, [ORG United Airlines] said [TIME Friday] it has increased fares by [MONEY $6] per round trip on flights to some cities"

 ***Sequence labeling***: assigning each word in an input sequence a label so that the output sequence is the same length (word count) as the input sequence.

***Count nouns***: "Count nouns can occur in the singular and plural (goat/goats, relationship/relationships) and can be counted (one goat, two goats)"
***Mass nouns***: " Mass nouns are used when something is conceptualized as a homogeneous group. So snow, salt, and proper noun communism are not counted (i.e., *two snows or *two communisms)"

***POS Tagging***: "The process of assigning a part-of-speech to each word in part-of-speech tagging a text".

Ex: "There are 70 children" -> "There/PRO/EX are/VERB/VBP 70/NUM/CD children/NOUN/NNS"

"Tagging is a ***disambiguation*** task; words are ***ambiguous*** — have more than one possible part-of-speech—and the goal is to find the correct tag for the situation. For example, book can be a verb (book that flight) or a noun (hand me that book). The goal of POS-tagging is to resolve these ambiguities, choosing the proper tag for the context."

There are different methods for selecting the correct tag. One common baseline for selecting and evaluating a tag selector is the ***Most Frequent Class Baseline***: When a word has the posibility for multiple tags, always choose the most common tag. This way we have something to compare a tag classifier too (performance wise) and make sure that for it to be passable, it has to be at least as good as the most frequent class classifier. 

Common issue with NER is that we are not labeling each word in a text, instead only some words are labeled (spans of text instead of words). Issue with this is that now we need to indentify which words are an entity and which arent.

One standard approach for sequence labeling is ***BIO Tagging***. This is a method that adds additional labels to each word that seperates words into spans of interest (a sequence capturing a entity) and outside of the span of interest. Tokens that start a span of interest are labeled with "B", tokens inside a span of interest "I" and tokens outside a span "O". With distinct "B" and "I" tags for each different entity labeled.

    Ex: "[PER Jane Villanueva ] of [ORG United] , a unit of [ORG United Airlines Holding]" -> Jane: B-PER, Villanueva: I-PER, of: O, United: B-ORG, etc..

Can also use ***IO Tagging*** which is a simplified version that drops the "B" tag and a more informative version ***BIOES Tagging*** that adds a "E" end tag.

***Markov Chain***: A model that returns info about the probability of a random variable (or state) based only on the current state. Ex: The probability of some type of weather occuring tomorrow based only on the current weather today. Formally Markov Assumption: P(qi=a | q1, ..., qi-1) = P(qi=a | qi-1) (ie. only the previous event matters for prediciting the future event) (this represents a Bi-gram model)

"Two common approaches to sequence modeling are a generative approach, HMM tagging, and a discriminative approach, CRF tagging."

***Hidden Markov Model (HMM)***: Markov model that can compute the probability of observable and unobservable events. For example, Using markov model to predict the next word is an observable event (since we can see the next word) versus predicting the next words tag is an unobservable event (since we dont see a tag for each word we read). The task of determining these hidden variables corresponding to an input sequence (thus outputting a corresponding hidden sequence) is called ***decoding*** or HMM decoding.

Issue with HMM is that it needs a lot of specific conditions to acheive high accuracy. One issue it faces is running into new or unknown words. Adding in general language rules that a person is familiar with (like "words starting with capital letters are likely to be proper nouns, words ending with ed tend to be past tense") is difficult to add in cleanly into HMM. We need to look into a model that has a method for adding in features like a linear logistic regression model.

***Conditional Random Field (CRF)***: "discriminative sequence model based on log-linear models".

We can hand create features to plug into our CRF to help with unknown words. One of the most important and popular features is ***word shape*** features. These take an unkown word and mask it to letters by some formula.
    Example: "mapping lower-case letters to ‘x’, upper-case to ‘X’, numbers to 'd', and retaining punctuation. Thus for example I.M.F. would map to X.X.X. and DC10-30 would map to XXdd-dd"

We can also add in our knowledge into training by inputting hand-made features. Some examples are ones that will let us know if we have a "-ed" ending (suffix) or a "we" prefix for example. "well-dressed" might generate the following functions: prefix(xi) = w, prefix(xi) = we, suffix(xi) = ed, suffix(xi) = d, word-shape(xi) = xxxx-xxxxxxx, short-word-shape(xi) = x-x. These can help a model find a pattern from training that included at least some of these functions if it hasnt seen "well-dressed" in the training set (it might have seen "well" and "dressed" before but the combination of the two using "-" it hasnt, but maybe it has seen patterns of words combined with "-" in the training and can extract some sort of pattern from that as well). Thus before training we create all our custom functions then see how often they occur for each word in our training set and use those to find patterns to understand unknown words.

One notable and often used function is the ***gazetter*** function. This function contains a list of millions of locations and simply returns 1 or 0 if the word is one of these locations. Basically a simple known location tagger, ex: gazetter("United States Census Bureau") -> 1.

"Part-of-speech taggers are evaluated by the standard metric of accuracy. Named entity recognizers are evaluated by recall, precision, and F1 measure."

Neural or CRF approaches are often the standard for academic research in entity tagging, but in commericial approaches NER is often based on a programmed combinations of lists and rules (due to ease of use, cost, etc..). 
Here is an example of a rule based iterative algorithm for entity tagging:

    1. First, use high-precision rules to tag unambiguous entity mentions. (Only predict the entities the model is very confident in)
    2. Then, search for substring matches of the previously detected names. (Grab substrings from found entities (we are confident these substrings belong to entity names)
    3. Use application-specific name lists to find likely domain-specific mentions. (Grab further entities but only popular common ones like location names (ex: New York))
    4. Finally, apply supervised sequence labeling techniques that use tags from previous stages as additional features. (Use all previously labeled entities as training data for some model and use to find more entities)
    5. Repeat

Note: Many languages have a much more rich morphology than english. Languages like Czech, Hungarian, and Turkish. Thus there are many more substrings that contain more meaning than in english. An example of this is languages that contain gendered cases (like masculine, feminine, neutured), genative, nomative, and accusative case. This causes our tagging models to have a much larger list of possible tags to choose from when classifying(tagging) words. Thus adjustments need to be made for models on different languages.

# Chapter 9 - RNNs and LSTMs

***Recurrent Neural Network (RNN)***: Same as a Neural network strucutre but has a recurrent feature that feeds in past weight values from a hidden layer at a previous feedforward pass. Each forward pass through a RNN has a timestep value t. At a hidden layer with a recurrent feature we take the input + the weights from the hidden layer at timestep t-1 multiplied by the weights of the hidden layer: x1 + (h1_ti-1)*h1_t1. The weights from the hidden layer at a previous timestep serves as a form of memory for the neural network. There is no fixed length on the recurrent weight thus the context embodied in the previous weight can stretch back to the information/words at the start of the input sequencet

    Feedforward function:
        h0 ←0
        for i←1 to LENGTH(x) do
            h_i ← g(U*hi_−1 + W*x_i) 
            y_i ← f(V*h_i)
        return y

U is the weight matrix for the previous hidden layer, W is the weight matrix for the input, V is the weight matrix for the hidden layer that gets us the output.

The forward pass for RNN is pretty simple, but for backpropagation it gets a bit more complex. This is because the output and weights from a feedforward pass at timestep t affects the output of the model at timestep t and timestep t+1. Since the output at timestep t+1 affects the output at timestep t+1 and t+2 we can conclude that the output at timestep t affects the ouput at timestep t+1, t+2, and so forth. Therefore we need to know the error the model had at timestep t-1 to get a proper reading of the error of the model at timestep t (as the error from the model at timestep t-1 will be included in the error of the model at timestep t). The process for accounting for these errors when updating the weights is called ***backpropagation through time***: where we backpropogate through each timestep and each time we save the error term for each weight at that specific timestep. We can then use the saved error from the previous timestep backpropagation to get a proper reading for the error at the current timestep of backpropagation

Forward Inference using word embeddings:

    e_t = E*x_t
    h_t = g(U*h_t−1 +W*e_t)
    y_t = softmax(V*h_t)

X = [x1;...;xt;...;xN] each x_i (or x_t) is a one hot vector representing a word with the size |V| x 1
We then multiply x_t by the embedding matrix E to get the dense embedding for the word.
Then we multiply the embedded word by the weight matrix W and add the weight matrix of the previous timestepped hidden layer multiplied by the value of the previous timestepped hidden layer. Then add an activation value on it to move the value outside of the linear zone.
V is the weight matrix we multiply the hidden layer output value by. 
"V*h can be thought of as a set of scores over the vocabulary given the evidence provided in h"
We then apply softmax over V\*h to normalize these scores into a probability value between 0 and 1.

During Training the model outputs a probability distribution of possible next words. We use self-supervision on a sequence of words to train the model. Each next word is represented as a one-hot vector of the word that represents the probabilities as well (every other word should be 0% likely while the correct word is 100% likely). We thus measure the loss using cross entropy which is used to measure the difference between two probability distributions.

Since we are doing next word prediction we have to take in the previous sequence of words so the model can use them to predict the next word: P(w_t | w1:t-1). In this scenerio we have just predicted w_t-1 and have taken the loss from it. In order to properly predict the following word (w_t) we need to make sure all the previous words are correct (as are from the orignal dataset we input) and there is a chance that our previous prediction was wrong. So, we need to make sure to put in the correct original history sequence of words. This is called ***Teacher forcing***. We are forcing the correct previous word sequence/history, even if the previous word prediction is wrong.

Text classification with RNN:

    1. We combine a RNN with a classification head with softmax layer for probabilities.
    2. We input our word sequence into the RNN one word at a time. Meaning at each timestep we input one word. Then the last hidden layer of our RNN will contain all the info from each of our words (since each timestep adds in the weights/info from the previous timestep, which was the previous word).
    3. We then pass this last hidden layer (h_n) to our classification head which turns it into a probability for the classification of the entire text. We take this output, calculate the loss (using cross entropy for probability dist), and pass it back throughout the entire network (including each timestep of the RNN) performing backpropagation.

"The training regimen that uses the loss from a downstream application to adjust the weights all the way through the network is referred to as ***end-to-end training***."

\*Note: Instead of using final hidden layer of RNN can use some form of pooling function on all the hidden states to capture the entire sequence of input text instead. Something like taking the sum of all hidden states or the max value at each column.*

***Autoregressive Generation (aka Causal LM Generation)***: The process of using some language model to incrementally generate words by sampling the next word conditioned on the previous words already generated. Ex: n-gram word sampling argmax P(word_i=x | "The cat").

This ***text generation*** technique can be a used using RNN in the following way:

    "
    1. Sample a word in the output from the softmax distribution that results from using the beginning of sentence marker, <s>, as the first input.
    2. Use the word embedding for that first word as the input to the network at the next time step, and then sample the next word in the same fashion.
    3. Continue generating until the end of sentence marker, </s>, is sampled or a fixed length limit is reached.
    "

One model strucutre we can make is to stack RNN's. This has shown increased improvment on many tasks as the different layers of RNN's can represent different levels of abstractions/details in the task (like the different layers in a regular Neural Network). This comes at a tradeoff of increased cost with each new layer of RNN added.


If we have access to the entire sequence of text at once we can perform something called ***Bidirectional RNN's***. These are two seperate RNN's that are trained from the left to right and from right to left (both ways). This can help with certain NLP tasks where both sides of text (or the entire length of text) provides a greater benefit to some prediction than just gathering data from one side. One note is that in a left to right RNN the hidden layer on the last word (that contains all the info/combined weights) from all the previous words, will have an emphasis (stronger memory) of the most recent words it was inputted (hence the last words). Combining hidden layers from a forward and backward RNN will even out the weight distribution.


***Long Short-Term Memory (LSTM)***: Module structure that utilizes the RNN along with a context layer that acts as a gate that controls the flow of information in and out of the models layers (memory managment). Ontop of the RNN we have 3 gates that filter our weights/information:

***Forget Gate***: Removes information from the context that is no longer needed. (What info is no longer useful and we should forget) ( (\*e*) represents element-wise vector multiplication, σ() = sigmoid_func())

    1. First takes a weighted sum of the previous hidden layer and the input passed through a sigmoid: f_t = σ(Uf*h_t-1 + Wf*x_t)
    2. The context mask is then applied element wise to remove the unwated information/weights: k_t = c_t−1(*e*)f_t
    3. We removed information from the previous layer, now we need to extract the rest of the information (that we want to keep) and combine it with our current input: g_t = tanh(Ug*h_t−1 +Wg*x_t)

***Add Gate***: Creates the mask for which information to add to the model and adds it into the current context (what info it needs to remember for the future).

    1. Create the mask for which of the weights we will add to the current context: i_t = σ(Ui*h_t−1 + Wi*x_t)
    2. Mask in the information we want to add with element wise multiplication: j_t = g_t(*e*)i_t
    3. Add our remaining information to our context vector: c_t = j_t + k_t

***Output Gate***: Masks and grabs the information we require right now for our current hidden state. (What info are we using right now for our current prediction)

    1. Create the mask for grabbing the current information needed from the current hidden layer and input: o_t = σ(Uo*h_t−1 +Wo*x_t)
    2. Apply the mask to our current context of data to grab our required information for prediction: ht = o_t(*e*)tanh(c_t)

So the steps are: Run the forget gate to forget our old uneeded info, then run the add gate to add new data to our memory. Finally, run the output gate to grab the info we need currently (a combintation of old and new info) to make a prediction.
"an LSTM accepts as input the context layer, and hidden layer from the previous time step, along with the current input vector. It then generates updated context and hidden vectors as output"

***Encoder-decoder network***: Network strucutre that consists of three components, an encoder, a context, and a decoder. The encoder takes in a text sequence as an input then captures and stores the relevant information inside the context. The decoder then takes the info from the context and generates a text output at an unspecified varying text length. Often used for tasks such as summarization, question answering, and machine translation. (Meant to be used when we are not mapping each word to a class or an entire text sequence to a class instead generating some type of response that doesnt have a specified length).


***Example of a simple encoder-decoder network model*** (using RNN's for machine translation):

    Encoder: The encoder network is a simple RNN that takes in each word in a sequence and outputs the last hidden layer as the context portion (since it contains data from all the words)

    Decoder: The decoder network is also a simple RNN but differs in that each iteration of the RNN has a task of predicting a word that will be part of the translated sentence from the original input sequence. Each iteration takes in the context (so it has the entire sequence from the encoder that it needs to translate), the previous hidden layer (so that it knows the info the last iteration had and what it predicted with it), and the word that the previous iteration predicted (so that it knows what word to predict next). Note: The first iteration of the decoder takes in a "end-of-sentence-tag" as the previous prediction of the decoder RNN.

    For training we input a tuple pair of the original stentence and the language translation (with an end of sentence token appeneded to the end of both sentences) Ex: ("The green witch arrived<s>", "llego la bruja verde<s>"). Then we take the cross entropy loss from each iteration of the decoder RNN of its word prediction probabilities and the actual probability of words it should output (one hot encoded of the target word). For each prediction we also perform teacher forcing by giving each iteration the correct previous word predicted (even if the model didnt predict it).

Issue with Encoder-decoder RNN model is that the context is the last hidden state of the RNN encoder which acts as a bottleneck since we need it to evenly represent the entire sentence and often the last hidden state is biased towards the most recent inputs/words. This can be solved with the ***attention mechanism***.


***attention mechanism***: Creates a context vector that changes at each iteration (word output) of the decoder, aka a dynamic context vector. This context vector has access to each of the hidden states from the encoder (therefore removing the last hidden state only bottleneck). This mechanism takes a list of hidden states (of any size, as will have different length of input sequences with different amount of hidden states for different inputs) and maps them to a fixed size context vector. This context vector has different weights of importance (from 0 to 1) assigned to each of the hidden states from the encoder. It calculates these weights by scoring how similar each of the encoder hidden states are to the decoder hidden state being passed in from the previous iteration. 
One iteration from decoder example:

    Imagine have 3 word input: "I am bob<s>" (which counts as 4 inputs since we append end of sentence tag) x1, x2, x3, x3 = ["I", "am", "Bob", "<s>"]
    
    Our encoder model is thus 4 hidden states H_e_1, H_e_2, H_e_3, H_e_4. 
    
    Imagine we are on output number i in the decoder process, thus we are taking input from the output and the hidden state of the last iteration of the decoder RNN: y_i-1 and H_d_i-1. As well as the context of the current information we have from the entire input: c_i.
    
    The output of decoder on iteration i is a softmax probability on a function that takes in the previous hidden state and input of the decoder and the current context: y_i = softmax(f(y_i-1, H_d_i-1, c_i))
    
    So far we are on the same process as a basic encoder-decoder network that uses the last hidden layer as the context. But how we utilize the attention mechanism here is that we update the context vector c at each iteration (hence the c_i).


    
    c_i is calculated as such:

    1. For each hidden layer of the encoder calculate the similarity between the encoder hidden layer and the previous decoder hidden layer: similarities(sims) = softmax([sim(H_e_1, H_d_i-1),  sim(H_e_2, H_d_i-1), sim(H_e_3, H_d_i-1), sim(H_e_4, H_d_i-1)]) = [0.4, 0.3, 0.1, 0.2]
    2. Apply these weights to each of the hidden layers in the encoder: c_i = [sims[0]*H_e_1, sims[1]*H_e_2, sims[2]*H_e_2, sims[3]*H_e_4]

    Now we have a relevent amount of attention to each of the hidden weights in the encoder (each of the words) based on the previous hidden weight of the decoder. (Based on the attention and input we used in the previous decoding step, what parts of the input data should we pay attention to now and by how much?).

    Note: The most basic similarity (sim) function we can use is simply a dot product between two hidden states (measuring similar dimensions) (H_e_j*.*H_d_i-1). But we can also use more advanced methods like: 
    
    "Instead of simple dot product attention, we can get a more powerful function that computes the relevance of each encoder hidden state to the decoder hidden state by parameterizing the score with its own set of weights, Ws. 
    sim(H_e_j, H_d_i-1) = (H_e_j*Ws*H_d_i-1)"

# Chapter 10 - Transformers and Large Language Models

***Attention intuition***: The point of attention is to be able to identify the true meaning of words with respect to their context (or the sentence they belong to). For example: "The ***chicken*** crossed the road because ***it*** wanted to get to the other side." We know that "it" refers to the chicken due to "The chicken" being established as the topic of the sentence.

***Casual self-attention (aka backward-looking self-attention)***: At each input and processing of a word from an input sequence, the model has access to that specific word and all the words that came before it. Each of the processing of attention on an input word and its preceding words is also done independently so that we can compute the attention of each word at the same time and utilize a GPU. Formally, let self-attention layer = sal() and an input "x1 x2 x3" where xi represents a word. We can compute the attention for each of the words like so at the same time: a1 = sal(x1), a2 = sal(x1, x2), a3 = sal(x1, x2, x3). 

***Formal (bare bones) self-attention layer computation***: Our goal for computing the self-attention of a targeted word is to find the relevance of its surrounding words in regards to this specific context. Ex: Based on the inputed "it" what is the relevance of "chicken" and "road" in this context (sentence) to "it". The computation is done as follows: 

    1. Each word (xi) is represented as a numeric token
    
    2. We use similarity to measure how relevent two words are to each other. In this case we can use simple dot product: sim(x1, x2) = x1*x2
    
    3. For each inputted word we calculate a probability value (between 0 and 1) for how similar two words are using softmax. This is done for all the previous words and the word itself: alpha_i_j = softmax(sim(xi, xj)) forAll j <= i. Ex: input x3: softmax([sim(x3, x1), sim(x3, x2), sim(x3, x3)]) = alpha_3_(j:1:3) = [0.2, 0.1, 0.7] (even though sim(x3, x3) will return largest num by far the softmax func will force the probabilities to sum to 1.0).
    
    4. We then take all the previous inputs along with the current input. Then use this to ouput an attention value that is the token values of each word (xi) multiplied by their probability of similarity to the target word as a weight: a_i = sum_[j<=i] alpha_i_j*x_j.
    Ex: attention for word 3: a3 = 0.2*x1 + 0.1*x2 + 0.7*x3  (where xi are numeric token values for each word)

This formal attention strucutre allows us to define key terms used in Transformers: query, key and value.

***Query***: The current focus of attention. The element we are comparing to all its previous elements and itself. (From the example above the query would be x3 (in point 3.)).

***Key***: The previous element being compared to the current focus of attention (query).

***Value***: The numeric value of the current word token. We multiply the ***value*** by the specific query-key softmax value to get the attention value. a = 0.3*(value)

To use these terms in a Tranformer we use weight matricies that we multiply by our input vector (x_i) to project the word into a query, key or value.

    q_i = x_i*W^Q - (1 x d) * (d x d_k) = (1 x d_k)

    k_i = x_i*W^K - (1 x d) * (d x d_k) = (1 x d_k)

    v_i = x_i*W^V - (1 x d) * (d x d_v) = (1 x d_v)

    Thus now our similarity computation is done as so: sim(xi, xj) = q_i * Transpose(k_j)
    and our attention value is calculated like so: sum_[j<=i] alpha_i_j * Transpose(v_i)

\*Note: One issue with our similarity function using dot product is that we can have very large values (from -inf to inf). Thus we need to scale it down. Usually this is done using some value based on the size of the word embeddings. One common method is to divide the similarity reading by the square root of the dimentionality of the query and key vectors: sim(xi, xj) = (q_i * k_i) / sqrt(d_k).*

Since we can compute our attention values independently we can compact our input into a matrix and perform simultaneous matrix multiplication to get our attention values as quickly as possible:

    Say we have N tokens (words) in a text sequence we then create the input matrix X of size (N x d). Now each row is one token from the input.
    We can then create our queries, keys, and values as result matricies: Q, K, V:

        Q = X*W^Q - (N x d) * (d x d_k) = (N x d_k)
        K = X*W^K - (N x d) * (d x d_k) = (N x d_k)
        V = X*W^V - (N x d) * (d x d_v) = (N x d_v)

        We can now compute all query and key value pairs at the same time using: Q*transpose(K) - (N x d_k) * (N x d_k) = (N x N)
        and our attention matrix A = self-attention(Q, K, V) = softmax((QK')/sqrt(d_k)) * V

\*Note: For casual attention we need to zero out the upper right triangle of the QK^T matrix as that side has the query-key values for words that are on the right side or the future words of the query.*

***Multi-head Attention***: There are many types of relationships between two words. "For example, distinct syntactic, semantic, and discourse relationships can hold between verbs and their arguments in a sentence". We cannot capture multiple complicated relationships with just one attention layer. Thus we create multiple attention blocks each with their own sets of weights (W^Q_i, W^K_i, W^V_i (for 1, 2, .., i, .., h)) to capture multiple relationships between words.

    1. We can run these attention heads simultaneously to obtain h amount of attenion head output weights: 

    h iterations simultaneously {
        
        Q = X*W^Q_i, K = X*W^K_i, V = X*W^V_i
    
        head_i = Self_Attention(Q, K, V)

    } => multi_head_attentions = [head_1, head_2, ..., head_h]


    2. We then concatinate the attentions values column wise: concatinated_heads = [head_1 + head_2 + head_3] - 3*(N x d_v) = (N x h*d_v)


    3. We then use a final linear projection matrix to project/reshape the concatinated output into our original attention matrix output: 
    
    concatinated_heads * W^O - (N x h*d_v) * (h*d_v x d) = (N x d)

For transformers a Multi-head attention layer is often used instead of the self-attention layer.

***Transformer Block***: A block that contains multiple layers of functions and networks that process text input. They usually consist of: A MultiHead Attention layer, Residual Connections, Normalization Layers, and FeedForward layers. each of these layers has some process in extracting information from the data and/or keeping our data within a certain constraint that allows the other layers to process it properly (wether it be through forward propogation or backward propogation).

***Residual Connections***: These simply append the original data to the output of said original data after it was passed through some layer or function. This is to help with backpropagation weight sizes as well as provide the model the original data along with how it was processed by the layer.

***Normalization Layer***: A normalization tool that keeps our data within a certain range so that multi layer forward and backward propagation doesnt result in any exploding (large) weights. A basic calculation method is to calculate the mean and standard deviation of the vector then apply them to the original vector so that the mean of the vector is 0 and the standard deviation is 1 (in terms of numerical values).

***Feedforward Layer***: contains N position-wise (neural) networks. Each of these networks has its own set of hidden layers and weights that take in processed information (like words that have attention calculated for their specific position and values normalized as well as residually added original info). Each of these networks calculates some non-linear relationship for the specific attentioned data at its position in the text sequence.

\*Note: In a ***prenorm transformer*** architecture (which often outperforms other transformer architectures) we add a Normalization layer at the very beginning before we pass in the data to our multi-head attention layer*.

***Positional Embedding***: When we input a text sequence into a transformer, we have an embedding vector of size (1xd) that represents each word. We would also like to include the information of the position of each word relative to its sentence. We obtain a (1xd) vector that represents the words position in terms of an index. Since this vector is the same size as our word embedding we simply sum these two vectors together as our "composite embedding" input. Ex: "Janet will back the bill" -> [Janet+1], [will+2], [back + 3], etc... (each of these is: 1xd + 1xd = 1xd).

***Language Model Head***: A block of additional layers we add ontop of our transformer model to extract its calculcated info (based on our input) and apply its knowledge to perform our task. For example, in word generation our language model head will contain a ***unembedding layer*** which takes the last token h_N from our last transformer layer and linearly projects the data into a distribution of 1x|V| vectors that represent likelihood values for each word in our vocabulary. We then pass these into a softmax function to get the probabilites for the next word (word N+1) from our list of words (vocabulary).

Note: We can train and learn this unembedding layer or more commonly, we use the transpose of the embedding matrix E so that this matrix can act as map dictionary that can convert one hot encoded vectors into word embeddings and convert word embeddings into "a vector over the vocabulary".

***Logit lens***: As a transformer interpretability method we can take the language model head and apply it to the token output for one of the intermediary transformer blocks to see what words each transformer block is representing/looking at. (An inside look on how its thinking).

** conditional generation (next word prediction based on previous words) is utilized extremely well by transformers as they can not only have access to the entire context of previous words at each prediciton (varies based on context window size) but also have access to their (the transformers) sequence of predicted words with respect to the given context. **

Since conditional generation performs so well with transformers people have modeled many other NLP tasks to be conditional generation problems. For example sentiment analysis can be done like so: Input: "The sentiment of the sentence: 'I like Jackie Chan' is:" then in the language model head have the output be modeled as: 

    P(positive| The sentiment of the sentence: 'I like Jackie Chan' is:) = ?
    P(negative| The sentiment of the sentence: 'I like Jackie Chan' is:) = ?

***Decoding***: The task of choosing the next word for a language model to predict based on the outputed probabilites of next possible words.

***Greedy Decoding***: At each timestep of next word prediction, always predict the word with the highest probability. This looks at only what is locally optimal and not the best choice globally for the entire sentence.

LLM's often dont use greedy decoding as then its resulting responses are extremely predictable, generic, and repetative.

One of the most common methods for decoding is ***sampling*** in which we create a distribution of all our next possible words according to their assigned model probabilities and sample from that distribution. This way our higher probabilty words have a higher chance to be picked but we arent gauranteed to pick the highest probability word.

We also need to avoid randomly sampling our entire word distribution as there are many odd words with low probabilities but since our word vocabulary is high we result in a large amount of low probability odd words that sum up to a large probability. This results in received odd (out of place) words in our generated sentence in some cases.

Thus there is a trade-off between quality of answers and diversity of answers. Words selected with the highest probabilities are often more accurate and quality, however they are very predictable and basic. While selection with more diversity can result in random giberish. There is therefore a middle ground weighting that holds accuracy while having more creativity and diversity in its answers.

One common method of sampling that is known as a "simple generalization of greedy decoding" is ***Top-k sampling*** where we limit our distribution to the top k words with the highest probabilities, then we normalize the probabilities to add up to 100% and randomly select from there. With this we can play with our trade off by changing our k value.

One issue with this is that since k is a fixed value it wont work well if our probability distribution of words changes sizes frequently. For example if our model returns 9 words than a top-10 sampling method will do nothing. We can combat this using ***Top-p sampling*** which instead uses the top p percentage of words instead of a fixed constant amount.

We can also use something called ***Temperature sampling*** in which we use a contantly decreasing temperature value that at the start has a high value (high temperature) which means its very flexible versus a low temperature it remains firm and in this case we use a low temperature value to increase the probabilities of our more confident probabilities. y = softmax(u/temp) with temp in (0, 1). When a high value is divided by a fraction its value increases which softmax pushes to 1 and others to 0. If we want to flatten the word distribution in terms of probability values we can set our temp value to be > 1.


***Scaling laws***: When training a LLM we can determine our performance by these 3 factors: number of non-embedding parameters (weights or layers), dataset size, and compute budget. The relationship with our model performance and these factors is a scaling factor that can be calculated with formulas. These formulas are useful in determing what modifications should we make to our model size, dataset size, computation/training time, to acheive a specific desired performance.

***Hallucination***: Language models are training to generate text that is coherent and logical, however there is no way for the model to verify that what its saying is true or not. When a model outputs something that is not true this is called a hallucination.

Some biases language models can have is generating toxic language that it learned from training text from banned or toxic websites. It can also have a bias towards outputing opinions based on the majority author group of its training data. For example, if most of our training data is english websites from the United States our model wont often output data from opinions on people from other countries.

Language models can also hold privacy risks of being able to leak personal information about someone that was in its training set. As well as generating copyrighted text that it was trained on. (In the united states for example, you are allowed to train a model on copyrighted text but the model can't generate or output this copyrighted text).

# Chapter 11 - Fine-Tuning and Masked Language Models

***Transfer learning***: Taking knowledge from one task or domain and then applying it to solve a new task. Common exmaple is taking a pretrained language model that already has rich understanding of word meaning and applying some kind of classifier head on top of it to use that knowledge to classify sentences into categories.

***Contextual embeddings***: each word is represented by a different vector depending on its context. Word embeddings from previous chapters had a constant vector value that represented the word (no matter what sentence or position it was in). (These constant vector embeddings obviously change and update as the model is being trained).

***Bidirectional Transformer Encoders***: Transformers that use an attention block that calculates the attention for each word using the entire sentence. This includes words that come after the targeted attention word. Since the model looks at both sides it can't really be used to produce running text and instead is used to take in a sentence and output a sentence of the same length hence encoding each word. This is also known as a ***Encoder-only*** model. While the models above that used left to right attention to produce running text are known as ***Decoder-only*** models.

The attention calculation follows the same as casual attention calculation with the only difference being that we dont need to zero out the upper right triangle of our attention matrix as we are using the context of the words to the right of our targeted attention word.

For bidirectional encoding we can no longer use our "predicted the next word" self-training method. Instead we implement a "fill in the blank" sentence for the model to predict (technicalled called the ***cloze task***). Ex: "Please turn ___ homework in" -> "Please turn your homework in".

***Masked Language Modeling***: The original method in training bidirectional encoders. In this method 15% of the word tokens are sampled for learning. Of those 15% samples, 80% are replaced with a physical [MASK] token, 10% are replaced with a randomly selected token, and the other 10% are left unchanged. Ex: "So long and thanks for all the fish" -> "So [MASK] and [MASK] for all apricot fish". We then predict each of these [MASK]'s and calculate the loss as the average cross entropy log loss between the two. Note that this method is inneficient as it only uses 15% of the words in our training data.

Other than word encoding (creating rich word representations), one important class of application for our Bidirectional transformer includes determing the relationship between pairs of sentences (this includes stuff like paraphrase detection, entailment (checks if entailing or contradicting sentences), or discourse coherence (if two sentences together form a coherent discourse (discussion)).

To learn this type of knowledge we perform ***Next Sentence Prediction (NSP)*** where we sample sentence pairs consisting of 50% positive samples and 50% negative samples (a sentence paired with a randomly selected sentence from elsewhere in the corpus). We then create a binary class prediction head for positive and negative and train the model this way. Ex: "[CLS] Cancel my flight [SEP] And the hotel [SEP]". (Cls is the beginnign tag and sep is the sentence seperator tag).

Now using these pre-trained transformers we can use their encoding output vector zi for word wi as the ***contextual embedding*** for word wi. We know that each vector represents that word in its specific context for its specific task. (Another common method is to average multiple zi's (using one zi from each layer or the last n layers for example) as the final embedding value). We can thus use these contextual embeddings for things like measuring the semantic similarity of two words in context and creating multiple different models for word meaning (in multiple languages as well).

***Word-sense***: A "discrete" or specific representation of one meaning of a word. For example 'mouse' can have mouse-word-sense-1: "small rodent" and mouse-word-sense-2: "hand held pointing device for a computer". 

***Word sense disambiguation (WSD)***: The process of selecting the proper word sense for a word given its context. 

To perform WSD one method is to first make a ***one sense per discourse*** assumption (which means a word will keep one word sense meaning throughout a discourse). Then we perform a 1-nearest-neighbor algorithm where we take a contextual embedding for each possible sense of a word then average it into a ***sense embedding***: v_s = (1/n)*sum_i v_i for_all v_i in tokens(s). Then at test time given a target word t, we take its contextual embedding along with its sense embedding and find the closest sense embedding using cosine similarity to find the most accurate word sense: sense(t) = argmax_(s in senses(t)) cosine(t, v_s).

One issue we have to be aware of in handling word embeddings is that the embeddings for each word tend to all lean towards ***anisotrpy***. Meaning that their cosign similarities (even if not similar at all) will be very close to 1. This is because some of the dimensions of word vectors tend to be much different in magnitude than the size of the values in other dimensions (known as ***rogue dimensions***. This causes a skew , one way to fix these dimensions is to standardize our vector (z-scoring). This is done by subtracting the mean and dividing by the standard deviation.

***Natural language inference (NLI) (aka recognizing textual entailment)***: Model predicts a Neutral, Contradicts, or Entails classification on a inputted pair of sentences (classifying the relationship between the pair of sentences).
    
    Examples: 
    "
    • Neutral
        a: Jon walked back to the town to the smithy.
        b: Jon traveled back to his hometown.
    • Contradicts
        a: Tourist Information offices can be very helpful.
        b: Tourist Information offices are never of any help.
    • Entails
        a: I’m confused.
        b: Not all of it is very clear to me.
    "

# Chapter 17 - Context-Free Grammars and Constituency Parsing

***Noun phrase***: A set of words that surround a noun. 
    Examples: "a high-class spot such as **Mindy’s**", "**the Broadway coppers** love..." -> a noun phrase can occur before a verb (some cant for ex: "**spot** sat")

***Context-free grammar (CFG) (a.k.a. phrase-structure grammar)***: A set of rules that provide a structure for ways in which categories/symbols of a language can be grouped and ordered together.

    "For example, the following productions NP express that an NP (or noun phrase) can be composed of either a ProperNoun or a determiner (Det) followed by a Nominal; a Nominal in turn can consist of one or more nouns."

    Noun phrase (NP) -> Det Nominal
    NP -> Proper Noun
    Nominal -> Noun | Nominal Noun (| = or)

    We can then combine the rules with each other and create a heirarchal order of conversions:

    Det -> A | the
    Noun -> Flight
    Result: the flight.

***Terminal symbols***: From CFG, words in a language. Ex: "Plane", "the", etc..

***Non-terminal symbols***: Symbols that represent abstractions over terminal symbols. Ex: "Det", "Noun", "Det Nominal". These non terminal symbols are used to assign terminal symbols to their lexicon category.
    

CFG can be used for generating sentences or assigning strucutre to a given sentence. Sentence creation example using rule expansion called a ***Derivation***: 

    Start with symbol: NP (noun phrase, meaning we want a noun phrase created)
    From NP we apply our first rules: Det Nominal
    Apply another rule to rewrite as: Det + Noun
    Result: "a flight"

Can also represent a derivation as a ***Parse-tree***: 
            NP
       ^Det - ^Nom
       ^a     ^Noun
              ^Flight


Can see here that NP **Dominates** all the nodes in the tree and that it **immediately dominates** Det and Nom.

***Start symbol***: In CFG each string is derived from a starting point S known as the start symbol.
Ex: S -> NP VP = "I prefer a morning flight". Creates a starting rule that shows that a noun phrase can be followed by a verb phrase.

***verb phrase***: Phrase in english that consits of a verb. One example is a verb followed by a noun phrase:

    VP -> Verb NP = "prefer a morning flight"

***Prepositional phrase***: An english phrase that contains a preposition which "shows direction, time, place, location, spatial relationships, or to introduce an object. Some examples of prepositions are words like 'in,' 'at,' 'on,' 'of,' and 'to.' ". "generally has a preposition followed by a noun phrase."

    PP -> Preposition NP = "from Los Angeles"

    VP -> Verb NP PP = "leave Boston in the morning"

***Tree bank***: A corpus where each sentence is annotated with a parse tree. Usually created by running a parser over each sentence then hand correcting each tree usually by human linguists.

***Formal language***: Possibly infinite set of strings of words.

***Grammatical sentences***: Sentences that can be defined by a formal grammer. Meaning we can represent a sentence by a grammer variable that is made up of other grammer words creating a parse tree. (can also be known as ***grammars***)

***Strongly equivalent grammars***: Both grammars generate the same set of strings and assign the same phrase structure to each sentence.

***Weakly equivalent grammars***: Both grammars generate the same set of strings but do not assign the same phrase structure to each sentence.

***Normal form for grammars***: A standard way of defining a set of rules that define a set of steps for replacing and rewriting symbols of a language to generate new strings that follow that languages grammar.